In [111]:
import numpy as np
import pandas as pd
import seaborn as sns
from pandas_datareader import data as wb
import nasdaqdatalink as nd

In [112]:
# CPIAUCSL: CPI 
# PPIACO: PPI(Commodities) 
# DFF: Federal Funds Effective Rate
# UMCSENT: Consumer Sentiement
# PAYEMS: Non-farm payroll
df = wb.DataReader(['CPIAUCSL', 'PPIACO', 'DFF', 'UMCSENT', 'PAYEMS'] , data_source='fred', start='1989-01-01')
df.columns = ['cpi', 'ppi', 'eff','sentiement', 'nonfarm']

In [113]:
df = df.resample('M').last()

In [114]:
from tvDatafeed import TvDatafeed, Interval
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

# Extract from Tradingview
driver = webdriver.Chrome('driver/chromedriver') 
tv = TvDatafeed()

#ISM
ism = tv.get_hist('MAN_PMI','ISM',interval=Interval.in_monthly, n_bars=5000)
ism = ism['close'].shift(1)
ism.columns = ['ism']

/var/folders/q5/4x95s775049742gq43bpyf7w0000gn/T/ipykernel_65006/2343914403.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('driver/chromedriver')


In [115]:
#SPY
spx = tv.get_hist('SPX500','FX',interval=Interval.in_weekly, n_bars=5000)
spx = spx['close']
spx.columns = ['spx']

In [116]:
#GOLD
gold = tv.get_hist('GOLD','TVC',interval=Interval.in_weekly, n_bars=5000)
gold = gold['close']
gold.columns = ['gold']

In [117]:
#USOIL
usoil = tv.get_hist('USOIL','FX',interval=Interval.in_weekly, n_bars=5000)
usoil = usoil['close']
usoil.columns = ['usoil']

In [118]:
# us10y
us10y = tv.get_hist('US10Y','TVC',interval=Interval.in_weekly, n_bars=5000)
us10y = us10y['close']
us10y.columns = ['us10y']

In [119]:
# us2y
us2y = tv.get_hist('US02Y','TVC',interval=Interval.in_weekly, n_bars=5000)
us2y = us2y['close']
us2y.columns = ['us2y']

In [120]:
# Store last price of month
df['ism'] = ism['1989-01-01':].resample('M').last()
df['spx'] = spx['1989-01-01':].resample('M').last()
df['gold'] = gold['1989-01-01':].resample('M').last()
df['usoil'] = usoil['1989-01-01':].resample('M').last()
df['us10y'] = us10y['1989-01-01':].resample('M').last()
df['us2y'] = us2y['1989-01-01':].resample('M').last()
df['yc'] = df['us10y'] - df['us2y'] 

In [124]:
# Label Recessions
df['label'] = 0
df.loc['1980-01':'1980-07', 'label'] = 1
df.loc['1981-07':'1982-11', 'label'] = 1
df.loc['1990-07':'1991-03', 'label'] = 1
df.loc['2001-03':'2001-11', 'label'] = 1
df.loc['2007-12':'2000-07', 'label'] = 1
df.loc['2020-02':'2020-04', 'label'] = 1

In [126]:
# Write CSV
df.to_csv("economic_indicators.csv")